In [22]:
# Import necessary packages and data
import csv
from string import letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division

#pandas-jupyter screen output settings
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [23]:
filepath=r"F:\machine learning\office\Ignite Backup\Akshay_MAC\akshay\Machine Learning\Learning Stuff\UOW\ML Foundations/amazon_baby.csv"
data=pd.read_csv(filepath,sep=",")
data=data.dropna()
data.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [24]:
#function to find count of selected_words.
def count_words(match_word,sentence):
    sentence=sentence.replace("\t"," ").replace("\n"," ").replace(".","").replace(",","")
    words = sentence.split(" ")
    x=0
    count = [x+1 for w in words if match_word == w]
    return sum(count);


In [25]:
#get word counts for each row for selected_words.
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

for word in selected_words:
    data[word]=data['review'].apply(lambda x:count_words(word,x))
data.shape

(182384, 14)

In [26]:
data.head()

,name,review,rating,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,0,0,0,0,0,0,0,0,0,0,0
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,0,0,0,0,1,0,0,0,0,0,0
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,0,0,0,0,0,0,0,0,0,0,0
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,0,0,0,0,2,0,0,0,0,0,0
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,0,1,0,0,0,0,0,0,0,0,0


In [27]:
# Answer 1: Most used word and least used from selected_words.
for word in selected_words: 
    print word, sum(data[word])

awesome 2611
great 46120
fantastic 1289
amazing 1823
love 34258
horrible 971
bad 4459
terrible 1028
awful 574
wow 88
hate 1108


In [28]:
#ignore all 3* reviews
data = data[data["rating"] != 3]
data.shape

(165679, 14)

In [29]:
data["sentiment"] = data["rating"] >=4
data.loc[data["sentiment"]==True,"sentiment"]=1
data.loc[data["sentiment"]==False,"sentiment"]=0
data.head()

,name,review,rating,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,0,0,0,0,1,0,0,0,0,0,0,True
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,0,0,0,0,0,0,0,0,0,0,0,True
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,0,0,0,0,2,0,0,0,0,0,0,True
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,0,1,0,0,0,0,0,0,0,0,0,True
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,0,1,0,0,0,0,0,0,0,0,0,True


In [30]:
X=data[selected_words]
y=data["sentiment"]

In [31]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=0)
logreg=LogisticRegression()
logreg.fit(X_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
print logreg.intercept_
zip([selected_words], logreg.coef_)

[ 1.42953831]


[(['awesome',
   'great',
   'fantastic',
   'amazing',
   'love',
   'horrible',
   'bad',
   'terrible',
   'awful',
   'wow',
   'hate'],
  array([ 0.96424813,  0.72414162,  0.76663809,  0.91477466,  1.33046089,
         -2.15804047, -0.94449396, -1.98036921, -1.84203578, -0.4358572 ,
         -1.35902839]))]

In [33]:
y_pred=logreg.predict(X_test)
y_pred
from sklearn.metrics import confusion_matrix

C = confusion_matrix(y_test,y_pred)
C

array([[  245,  4977],
       [  100, 27814]])

In [34]:
# error on Test Data Set

(245+27814)/len(y_test)

0.8467829550941575

In [35]:
#baseline accuracy(majority class classifier)
float(sum(y_test)/len(y_test))

0.8424070497344278

In [36]:
# explore a particular product's reviews
diaper_champ_reviews=data[data["name"]=="Baby Trend Diaper Champ"]
X_diaper_champ_reviews=diaper_champ_reviews[selected_words]
y_diaper_champ_reviews=diaper_champ_reviews["sentiment"]

In [37]:
y_pred_diaper=pd.DataFrame(logreg.predict_proba(X_diaper_champ_reviews))
diaper_champ_reviews = diaper_champ_reviews.reset_index(drop=True)
diaper_champ_reviews["est_prob"]=y_pred_diaper[1]

In [38]:
diaper_champ_reviews.sort(["est_prob"],ascending=False)

C:\Users\akxay\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,name,review,rating,awesome,great,fantastic,amazing,love,horrible,bad,terrible,awful,wow,hate,sentiment,est_prob
122,Baby Trend Diaper Champ,I received my Diaper Champ at my baby shower f...,5,0,0,0,0,3,0,0,0,0,0,0,True,0.995596
103,Baby Trend Diaper Champ,I love this diaper pale and wouldn\'t dream of...,5,0,2,0,0,1,0,0,0,0,0,0,True,0.985346
168,Baby Trend Diaper Champ,I have a two-year-old son and I love the Diape...,5,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
58,Baby Trend Diaper Champ,"This is absolutely, by far, the best diaper pa...",5,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
220,Baby Trend Diaper Champ,I\'ve worked with kids more than half my life....,5,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
9,Baby Trend Diaper Champ,We had 2 diaper Genie\'s both given to us as a...,4,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
232,Baby Trend Diaper Champ,I love this diaper pail! It\'s so easy to use...,5,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
134,Baby Trend Diaper Champ,"Love it, love it, love it! This lives up to t...",5,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
131,Baby Trend Diaper Champ,I love this diaper pail. It keeps the diapers ...,4,0,0,0,0,2,0,0,0,0,0,0,True,0.983544
81,Baby Trend Diaper Champ,I love my Diaper Champ. My son is 3 months old...,4,0,1,0,0,1,0,0,0,0,0,0,True,0.970233


In [39]:
#Confusion Matrix Function

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')




